In [3]:

import sys
sys.path.insert(0, '/app')

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
import os
import matplotlib.patches as mpatches

from scenario_identification.utils.path_helper import get_dataset_path


BUILD_DEVICE='local'

dataset_path = get_dataset_path(BUILD_DEVICE)

csv_dataset =  os.path.join(dataset_path, 'ml_iot-unsw.csv')


df_dataset = pd.read_csv(csv_dataset)


# def filter_self_loops(df_dataset):
#     mask = df_dataset['u'].values != df_dataset['i'].values
#     df_filtered = df_dataset[mask]
#     df_dataset = df_filtered
#     return df_dataset
    
# df_dataset = filter_self_loops(df_dataset)


def visualize_graph(G, df_time_slice, node_positions, time):
    # plt.figure(figsize=(24,24))
    
    # colors = [(0.6, 0.8, 1) if label == 0 else (1, 0.6, 0.6) for label in target_labels]

    # nx.draw_networkx(G, pos=node_positions, with_labels=True,
    #                  node_color=colors, cmap="Set2")
    
    # edge_attributes = nx.get_edge_attributes(G,'edge_attr')
    
    # nx.draw_networkx_edge_labels(G, pos=node_positions, edge_labels=edge_attributes)    
    # nx.draw_networkx_edges(G, pos=node_positions, edge_color='black', arrows=False)
    

    # plt.show()
    plt.figure(figsize=(24,24))
    
    # Node colors based on type
    node_colors = []
    for node in G.nodes():
        row = df_time_slice[df_time_slice['u'] == node].iloc[0]
        if row['type_EMERGENCY_VEHICLE']:
            node_colors.append('red')
        elif row['type_INDUCTION_LOOP']:
            node_colors.append('green')
        elif row['type_SMART_PHONE']:
            node_colors.append('blue')
        elif row['type_TRAFFIC_CAMERA']:
            node_colors.append('purple')
        elif row['type_TRAFFIC_LIGHT']:
            node_colors.append('yellow')
        elif row['type_VEHICLE']:
            node_colors.append('orange')
        else:
            node_colors.append('grey')
        
    # Node shapes based on status
    node_shapes = []
    for _, row in df_time_slice.iterrows():
        if row['shape_status_DEFORMED']:
            node_shapes.append('o')
        elif row['shape_status_DENTS']:
            node_shapes.append('s')
        elif row['shape_status_HEAVILY_DAMAGED']:
            node_shapes.append('v')
        elif row['shape_status_ORIGINAL_MANUFACTURED']:
            node_shapes.append('^')
        elif row['shape_status_SCRATCHES']:
            node_shapes.append('<')
        else:
            node_shapes.append('>')
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos=node_positions, node_color=node_colors, node_shape='o')
    
    # Draw another circle around nodes based on shape_status
    for shape, marker in zip(set(node_shapes), ['o', 's', 'v', '^', '<', '>']):
        nx.draw_networkx_nodes(G, pos=node_positions, nodelist=[node for node, shape_ in zip(G.nodes(), node_shapes) if shape_ == shape], node_shape=marker, node_color='none', linewidths=2)
    
    legend_labels = [
        mpatches.Patch(color='red', label='Emergency Vehicle'),
        mpatches.Patch(color='green', label='Induction Loop'),
        mpatches.Patch(color='blue', label='Smart Phone'),
        mpatches.Patch(color='purple', label='Traffic Camera'),
        mpatches.Patch(color='yellow', label='Traffic Light'),
        mpatches.Patch(color='orange', label='Vehicle'),
        mpatches.Patch(color='grey', label='Other')
    ]
    plt.legend(handles=legend_labels)
    
    
    
    # Draw another circle around nodes where label = 1
    label_one_nodes = [row['u'] for _, row in df_time_slice.iterrows() if row['label'] == 1]
    nx.draw_networkx_nodes(G, pos=node_positions, nodelist=label_one_nodes, node_color='none', linewidths=2, edgecolors='black')
    

    edge_colors = ['black' if row['post_label'] > 0 else 'red' for _, row in df_time_slice.iterrows()]
    nx.draw_networkx_edges(G, pos=node_positions, edge_color=edge_colors)
    
    plt.text(160, 160, f'Time: {time}', fontsize=12)
        
    plt.xticks(range(150, 1101, 100))
    plt.yticks(range(150, 1101, 100))
    
    plt.xlim(150, 1100)
    plt.ylim(150, 1100)
    
    plt.axis([150, 1100, 150, 1100])
    
    plt.savefig(f"graph_{time}.png")
    # plt.show()
    plt.close()
    


In [4]:

unique_timesteps = df_dataset['ts'].unique()

def get_node_positions(df_time_slice):
    node_positions = {}
    for index, row in df_time_slice.iterrows():
        node_positions[row['u']] = (row['latitude'], row['longitude'])
    return node_positions

def add_edge_attributes(df_time_slice, G):
    for index, row in df_time_slice.iterrows():
        u, v = row['u'], row['i']
        distance = row['distance']
        # Only add edges for nodes that are in df_time_slice
        if u in node_positions and v in node_positions:
            G.add_edge(u, v, edge_attr=distance)
    return G

print(f"Number of unique timesteps: {len(unique_timesteps)}")

for time in range(1, 200):
    df_time_slice = df_dataset[df_dataset['ts'] == time]
    
    G = nx.Graph()
    
    node_positions = get_node_positions(df_time_slice)
    G = add_edge_attributes(df_time_slice, G)
    
    G.remove_edges_from(nx.selfloop_edges(G))
    
    missing_nodes = [node for node in G.nodes() if node not in node_positions]
    if missing_nodes:
        print(f"Skipping timestep {time} due to missing positions for nodes {missing_nodes}")
        continue
    
    visualize_graph(G, df_time_slice, node_positions, time)
    print(f"Finished timestep {time}")


Number of unique timesteps: 1334


: 

: 